# AOC2019 - Day 18

Today's challenge: [Day 18](https://adventofcode.com/2019/day/18)

Data are manually downloaded into `./data/input_18.txt`

In [1]:
with open('./data/input_18.txt') as f:
    raw = f.read()

In [2]:
import networkx as nx

from itertools import combinations
from collections import defaultdict

In [3]:
# prepare labyrinth
rows = raw.strip().split("\n")
edges = []
keys = dict()
doors = dict()
start = (None, None)

for y, row in enumerate(rows[:-1]):
    for x, c in enumerate(row[:-1]):
        if c != '#':
            if c == '@':
                start = (x,y)
            if c.islower():
                keys[c] = (x,y)
            if c.isupper():
                doors[c] = (x,y)
            if row[x+1] != '#':
                edges.append(((x,y), (x+1,y)))
            if rows[y+1][x] != '#':
                edges.append(((x,y), (x,y+1)))
                
doors_ = set(doors.values())
backdoors = {v: k.lower() for k, v in doors.items()}
G = nx.Graph(edges)
# Since it's (almost) a tree, there is only one path from key a to key b.
# So it cannot be optimized with further keys collected
G.number_of_edges(), G.number_of_nodes()

(3204, 3201)

In [4]:
distances = defaultdict(dict)
obstacles = defaultdict(dict)

for x, y in combinations(keys, 2):
    p = nx.shortest_path(G, keys[x], keys[y])
    distances[x][y] = len(p)-1
    distances[y][x] = len(p)-1
    if doors_.intersection(p): # collect eventual obstacles, by name!
        obstacles[x][y] = {backdoors[d] for d in doors_.intersection(p)}
        obstacles[y][x] = {backdoors[d] for d in doors_.intersection(p)}

for x in keys:
    p = nx.shortest_path(G, start, keys[x])
    if not doors_.intersection(p):
        distances['@'][x] = len(p)-1

In [5]:
class History:
    def __init__(self, hist=None, d=0, distances=dict(), obstacles=dict()):
        self.d = d
        self.distances = distances
        self.obstacles = obstacles
        if hist is None:
            self.hist = ['@']
        else:
            self.hist = hist
        self.used_keys = set(['@'])
        self.res = []
    
    def get_key(self):
        ks = "".join(sorted(self.hist))
        last_ks = self.hist[-1]
        return ks, last_ks
    
    def next_keys(self):
        res = []
        k = self.hist[-1]
        for k_, d_ in self.distances[k].items():
            if k_ in self.used_keys:
                continue
            elif k_ in self.obstacles[k] and obstacles[k][k_].difference(self.used_keys):
                continue
            else:
                res.append((k_, self.d + d_))
        return sorted(res, key=lambda x: x[1])
        
    def add_step(self, step):
        k, d = step
        hist_ = self.hist[:]
        hist_.append(k)
        new_hist = History(hist=hist_,distances=distances,obstacles=obstacles)
        new_hist.d += d
        new_hist.used_keys = set(self.used_keys).union([k])
        return new_hist
    
    def backtrack(self):
        global all_time_best, memo
        k = self.get_key()
        if k in memo and self.d >= memo[k]:
            return []
        else:
            memo[k] = self.d
        next_steps = self.next_keys()
        if len(next_steps) == 0: # all keys collected
            if all_time_best is None:
                all_time_best = self.d
                print("First result: ", all_time_best)
                print(self.hist)
            if self.d < all_time_best:
                print("New best!", self.d)
                all_time_best = self.d
                return [self.d]
        # continue exploration
        for step in next_steps:
            new_hist = self.add_step(step)
            self.res += new_hist.backtrack()
        return self.res

In [6]:
all_time_best = None
memo = dict()
h = History(distances=distances, obstacles=obstacles)
h.backtrack()[-1]

First result:  5762
['@', 'j', 'r', 'v', 'y', 'p', 'm', 'q', 's', 'x', 'a', 'u', 'b', 'd', 'o', 'e', 'w', 'c', 'k', 'i', 'l', 'h', 'z', 'g', 'n', 'f', 't']
New best! 5758
New best! 5426
New best! 5242
New best! 5226
New best! 5194
New best! 5190
New best! 5182


5182

In [7]:
# Part 2, split in 4 labs, at line&row 40

In [8]:
# splitting into quadrants
w, h = 40, 40 # different w/h because tests where also rectangular
labs = [
    [row[:w+1] for row in rows[:h+1]],
    [row[w:] for row in rows[:h+1]],
    [row[:w+1] for row in rows[h:]],
    [row[w:] for row in rows[h:]]
]

starts = [
    (0,w-1,h-1),
    (1,1,h-1),
    (2,w-1,1),
    (3,1,1)
]
edges = []
keys = dict()
doors = dict()

In [9]:
# prepare graph
for i, rows_ in enumerate(labs):
    for y, row in enumerate(rows_[:-1]):
        for x, c in enumerate(row[:-1]):
            if c != '#':
                if c.islower():
                    keys[c] = (i,x,y)
                if c.isupper():
                    doors[c] = (i,x,y)
                if row[x+1] != '#':
                    edges.append(((i,x,y), (i,x+1,y)))
                if rows_[y+1][x] != '#':
                    edges.append(((i,x,y), (i,x,y+1)))

doors_ = set(doors.values())
backdoors = {v:k.lower() for k, v in doors.items()}
G = nx.Graph(edges)
G.number_of_edges(), G.number_of_nodes()

(3204, 3207)

In [10]:
# precompute distances and obstacles
distances = defaultdict(dict)
obstacles = defaultdict(dict)

for x, y in combinations(keys, 2):
    x_, y_ = keys[x], keys[y]
    if x_[0] != y_[0]:
        continue
    p = nx.shortest_path(G, x_, y_)
    distances[x][y] = len(p)-1
    distances[y][x] = len(p)-1
    if doors_.intersection(p): # collect eventual obstacles, by name!
        obstacles[x][y] = {backdoors[d] for d in doors_.intersection(p)}
        obstacles[y][x] = {backdoors[d] for d in doors_.intersection(p)}
        
for k, coords in keys.items():
    start = starts[coords[0]]
    p = nx.shortest_path(G, start, coords)
    distances['@%s' % coords[0]][k] = len(p)-1
    if doors_.intersection(p):
        obstacles['@%s' % coords[0]][k] = {backdoors[d] for d in doors_.intersection(p)}
#        distances['@%s' % coords[0]][k] = len(p)-1

In [11]:
class HistorySplit:
    def __init__(self, hist=None, d=0, distances=dict(), obstacles=dict()):
        self.d = d
        self.distances = distances
        self.obstacles = obstacles
        if hist is None:
            self.hist = [
                ['@0'],['@1'],['@2'],['@3']
            ]
        else:
            self.hist = hist
        self.used_keys = set(['@0','@1','@2','@3'])
        self.res = []
    
    def get_key(self):
        ks = "".join(sorted(sum(self.hist, [])))
        last_ks = "".join([xs[-1] for xs in self.hist])
        return ks, last_ks
    
    def next_keys(self):
        res = []
        for i, hist in enumerate(self.hist):
            k = hist[-1]
            for k_, d_ in self.distances[k].items():
                if k_ in self.used_keys:
                    continue
                elif k_ in self.obstacles[k] and obstacles[k][k_].difference(self.used_keys):
                    continue
                else:
                    res.append((k_, self.d + d_, i))
        return sorted(res, key=lambda x: x[1])
        
    def add_step(self, step):
        k, d, i = step
        hist_ = [xs[:] for xs in self.hist]
        hist_[i].append(k)
        new_hist = HistorySplit(hist=hist_,distances=distances,obstacles=obstacles)
        new_hist.d += d
        new_hist.used_keys = set(self.used_keys).union([k])
        return new_hist
    
    def backtrack(self):
        global all_time_best_, memo
        k = self.get_key()
        if k in memo and self.d >= memo[k]:
            return []
        else:
            memo[k] = self.d
        next_steps = self.next_keys()
        if len(next_steps) == 0: # all keys collected
            if all_time_best_ is None:
                all_time_best_ = self.d
                print("First result: ", all_time_best_)
                return [self.d]
            if self.d < all_time_best_:
                print("New best!", self.d)
                all_time_best_ = self.d
                return [self.d]
        # continue exploration
        for step in next_steps:
            new_hist = self.add_step(step)
            self.res += new_hist.backtrack()
        return self.res

In [12]:
all_time_best_ = None
memo = dict()
h = HistorySplit(distances=distances, obstacles=obstacles)

In [13]:
h.backtrack()[-1]

First result:  2154


2154